### Configuração inicial

In [1]:
import sys
sys.path.append('D:\\__case_ifood\\notebooks\\utils') # Defina aqui a pasta

In [ ]:
import importlib
import utils

# Recarregar o módulo 'utils'
importlib.reload(utils)

from utils import get_bq_client, create_dataset_and_table, credencial_gcp, pasta_projeto, bucket_name, os, bigquery, service_account, storage, pl, pd,  gc, tarfile, io


In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credencial_gcp

# Define o diretório para salvar os arquivos baixados do Bucket
diretorio_arquivos_bucket = os.path.join(pasta_projeto,"bases_dados")

# Nome do bucket e do projeto
bucket_name = "case_ifood_fsg"
#id_projeto = "case-ifood-fsg"

client = storage.Client()

client_bq = get_bq_client() #bigquery.Client(project=id_projeto)


### Para o arquivo order.json.gz não foi possível fazer o insert com processamento local por restrições de memória, com isso, foi criado a tabela diretamente no BQ na camada bronze chamada orders.

### Carrega o arquivo restaurant.csv na camada Bronze do BQ como 'merchants'

In [ ]:
# Define o nome do arquivo
file_name_merchants = "restaurant.csv.gz" 

# Acessa o bucket e o blob (arquivo)
bucket_merchants = client.bucket(bucket_name)
blob_merchants = bucket_merchants.blob(file_name_merchants)

# Faz o download como bytes
data_bytes_merchants = blob_merchants.download_as_bytes()

df_merchants = pd.read_csv(io.BytesIO(data_bytes_merchants), compression='gzip')

# Criando a coluna que indica quando o registro foi processado para ser salvo no BQ
df_merchants["insert_date"] = pd.Timestamp.today()

# Insere os dados no BQ
create_dataset_and_table(
    df=df_merchants,
    dataset_nome="bronze",
    tabela_nome="merchants",
    client=client_bq
)

del file_name_merchants
del bucket_merchants
del blob_merchants
del data_bytes_merchants
del df_merchants
gc.collect()


Dataset 'case-ifood-fsg.bronze' pronto.
Tabela 'case-ifood-fsg.bronze.merchants' criada com 7292 linhas.


495

### Carrega o arquivo consumer.csv na camada Bronze do BQ como 'consumer'

In [ ]:
file_name_consumer = "consumer.csv.gz" 

# Acessa o bucket e o blob (arquivo)
bucket_consumer = client.bucket(bucket_name)
blob_consumer = bucket_consumer.blob(file_name_consumer)

# Faz o download como bytes
data_bytes_consumer = blob_consumer.download_as_bytes()

df_consumer = pd.read_csv(io.BytesIO(data_bytes_consumer), compression='gzip')

# Criando a coluna que indica quando o registro foi processado para ser salvo no BQ
df_consumer["insert_date"] = pd.Timestamp.today()

# Insere os dados no BQ
create_dataset_and_table(
    df=df_consumer,
    dataset_nome="bronze",
    tabela_nome="consumer",
    client=client_bq
)

del file_name_consumer # Liberar memória
del bucket_consumer # Liberar memória
del blob_consumer # Liberar memória
del data_bytes_consumer # Liberar memória
del df_consumer # Liberar memória

gc.collect() # Coletar lixo para liberar memória

Dataset 'case-ifood-fsg.bronze' pronto.
Tabela 'case-ifood-fsg.bronze.consumer' criada com 806156 linhas.


437

### Carrega o arquivo ab_test_ref.csv na camada Bronze do BQ como ab_test

In [17]:
file_name_ab_test = "ab_test_ref.tar.gz" 

# Acessa o bucket e o blob (arquivo)
bucket_ab_test = client.bucket(bucket_name)
blob_ab_test = bucket_ab_test.blob(file_name_ab_test)

# Faz o download como bytes
data_bytes_ab_test = blob_ab_test.download_as_bytes()

# Salva o arquivo na máquina local
blob_ab_test.download_to_filename(os.path.join(diretorio_arquivos_bucket,file_name_ab_test))
print(f"Arquivo salvo em: {diretorio_arquivos_bucket}")

del bucket_ab_test # Liberar memória
del blob_ab_test # Liberar memória
del data_bytes_ab_test # Liberar memória

Arquivo salvo em: D:\__case_ifood\bases_dados


In [ ]:
nome_arquivo_leitura = "ab_test_ref.csv"

# Extrai tudo
with tarfile.open(os.path.join(diretorio_arquivos_bucket,file_name_ab_test), "r:gz") as tar:
    tar.extractall(path=diretorio_arquivos_bucket)

# Faz a leitura do arquivo que possui os dados
df_ab_test = pd.read_csv(os.path.join(diretorio_arquivos_bucket,nome_arquivo_leitura), encoding="utf-8")

# Criando a coluna que indica quando o registro foi processado para ser salvo no BQ
df_ab_test["insert_date"] = pd.Timestamp.today()

# Insere os dados no BQ
create_dataset_and_table(
    df=df_ab_test,
    dataset_nome="bronze",
    tabela_nome="ab_test",
    client=client_bq
)

del nome_arquivo_leitura # Liberar memória
del file_name_ab_test # Liberar memória
del df_ab_test # Liberar memória
gc.collect() # Coletar lixo para liberar memória

C:\Users\fsgon\AppData\Local\Temp\ipykernel_15152\1735237186.py:5: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=diretorio_arquivos_bucket)


Dataset 'case-ifood-fsg.bronze' pronto.
Tabela 'case-ifood-fsg.bronze.ab_test' criada com 806467 linhas.


1329